In [2]:
from dbPack import *
from basicPack import *

# conn = connect('clickhouse://192.168.1.42')
 
pd.read_sql('select * from 591_xgb.user_act limit 10',conn)

,COOKIE,USER_ID,ITEM_ID,SECTION_ID,EVENT_TYPE,EVENT_VALUE,TIMESTAMP,DEVICE
0,0,0,S8900040,12,click,1,1612281423,1
1,0,0,S8900040,12,stay,9464,1612281423,1
2,0,0,S8895773,12,click,1,1612281512,1
3,0,0,S8895773,12,stay,76663,1612281512,1
4,0,0,S8895541,12,click,1,1612281522,1
5,0,0,S8895541,12,stay,6446,1612281522,1
6,0,0,S8890917,12,click,1,1612281540,1
7,0,0,S8890917,12,stay,3933,1612281540,1
8,0,0,S8644155,41,click,1,1612503280,1
9,0,0,S8644155,41,stay,66014,1612503280,1


# user_act表

In [15]:
# pd.read_sql('DROP TABLE IF EXISTS  591_xgb.user_act ',conn)
create_sql = "\
CREATE TABLE 591_xgb.user_act \
( \
    `COOKIE` String, \
    `USER_ID` String, \
    `ITEM_ID` String, \
    `SECTION_ID` Int16, \
    `EVENT_TYPE` String, \
    `EVENT_VALUE` Int32, \
    `TIMESTAMP` Int32, \
    `DEVICE` Int8 \
) \
ENGINE = MergeTree() \
PRIMARY KEY COOKIE \
ORDER BY (COOKIE, TIMESTAMP)\
TTL toDateTime(`TIMESTAMP`) + toIntervalMonth(2)"

pd.read_sql(create_sql,conn)

""


# auto_view表

In [16]:
# pd.read_sql('DROP TABLE IF EXISTS  591_xgb.auto_view ',conn)
create_sql = "\
CREATE TABLE 591_xgb.auto_view \
( \
    `COOKIE` String, \
    `USER_ID` String, \
    `ITEM_ID` String, \
    `REGION_ID` Int8, \
    `SECTION_ID` Int16, \
    `TIMESTAMP` Int32, \
    `ACTION` Int16, \
    `CLICK` Int16, \
    `STAY` Int32, \
    `DIAL` Int16, \
    `COLLECT` Int16, \
    `FRIEND` Int16, \
    `SHARE` Int16, \
    `MESSAGE` Int16, \
    `PRINT` Int16, \
    `QUESTION` Int16 \
) \
ENGINE = MergeTree() \
PRIMARY KEY TIMESTAMP \
ORDER BY (TIMESTAMP, COOKIE, USER_ID) \
TTL toDateTime(`TIMESTAMP`) + toIntervalMonth(2)"

pd.read_sql(create_sql,conn)

""


# item表

In [18]:
pd.read_sql('DROP TABLE IF EXISTS  591_xgb.item ',conn)
create_sql = "\
CREATE TABLE 591_xgb.item \
( \
    `ITEM_ID` String, \
    `PRICE` Int32, \
    `UNIT_PRICE` Float32, \
    `MONTH_PAY` Float32, \
    `ROOM` Int8, \
    `HALL` Int8, \
    `TOILET` Int8, \
    `BALCONY` Int8, \
    `AGE` Int16, \
    `AREA` Float32, \
    `CONTAIN_AREA` Int8, \
    `POSTTIME` Int32, \
    `FLOOR` String, \
    `TOWARD` String, \
    `REGION` Int8, \
    `SECTION` Int16, \
    `HOUSE_TYPE` Int8, \
    `KIND` Int8, \
    `SHAPE` Int8, \
    `FITMENT` Int8, \
    `FEE` Int8, \
    `LEASE` Int8, \
    `LEGAL` Int8, \
    `PARKING_TYPE` Int8, \
    `PARKING_AREA` Float32, \
    `PUBLIC_RATE` Float32, \
    `NEAR_SHOP` Int8, \
    `NEAR_MARKET` Int8, \
    `NEAR_SUPER` Int8, \
    `NEAR_PARK` Int8, \
    `NEAR_SCHOOL` Int8, \
    `NEAR_HOSPITAL` Int8, \
    `NEAR_NIGHT_MARKET` Int8, \
    `NEAR_BUS` Int8, \
    `NEAR_TRAIN` Int8, \
    `NEAR_MRT` Int8, \
    `COMMUNITY` String, \
    `SCHOOL` String, \
    `BUSINESS` String, \
    `MRT` String, \
    `COMMUNITY` String, \
    `CREATED_AT` Int32, \
    `STATUS` Int8, \
    `CLOSED` Int8 \
) \
ENGINE = ReplacingMergeTree() \
PRIMARY KEY ITEM_ID \
ORDER BY (ITEM_ID) \
TTL toDateTime(`CREATED_AT`) + toIntervalMonth(3)"

pd.read_sql(create_sql,conn)

# # 加欄位 https://clickhouse.tech/docs/en/sql-reference/statements/alter/column/#alter_add-column
# pd.read_sql('ALTER TABLE 591_xgb.item ADD COLUMN IF NOT EXISTS `COMMUNITY` String AFTER MRT',conn)

""


# item_day表

In [13]:
# pd.read_sql('DROP TABLE IF EXISTS  591_xgb.item_day ',conn)
create_sql = "\
CREATE TABLE 591_xgb.item_day \
( \
    `ITEM_ID` String,\
    `DAY` Int32,\
    `CLICK` Int16,\
    `ACTIVE` Int16,\
    `VIEW`  Int16,\
    `CLICK_RATE` Float32,\
    `ACTIVE_RATE` Float32\
) \
ENGINE = MergeTree() \
PRIMARY KEY ITEM_ID \
ORDER BY (ITEM_ID, DAY) \
TTL toDateTime(`DAY`) + toIntervalMonth(3)"

pd.read_sql(create_sql,conn)

""


# recmd表

In [14]:
# pd.read_sql('DROP TABLE IF EXISTS  591_xgb.recmd ',conn)
create_sql = "\
CREATE TABLE 591_xgb.recmd \
( \
    `COOKIE` String,\
    `USER_ID` String,\
    `ITEM_ID` String,\
    `QUERY` String,\
    `TIMESTAMP`  Int32\
) \
ENGINE = MergeTree() \
PRIMARY KEY COOKIE \
ORDER BY (COOKIE, TIMESTAMP) \
TTL toDateTime(`TIMESTAMP`) + toIntervalMonth(2)"

pd.read_sql(create_sql,conn)

""


# 測試
### 結論:
https://clickhouse.tech/docs/en/sql-reference/statements/optimize/
去重複 <br>
MergeTree 用'optimize table 591_xgb.auto_view DEDUPLICATE' <br>
ReplacingMergeTree 用'optimize table 591_xgb.auto_view FINAL' <br>

In [586]:
insrt = "insert into 591_xgb.user_act values \
        ('A000','A000', 'code1', 1,'stay', 100, now()+5, 3),\
        ('A000','A000', 'code1', 1,'stay', 100, now()+5, 3), \
        ('A001','A001', 'code1', 1,'stay', 100, now()+5, 3);"
pd.read_sql(insrt,conn)

""


In [750]:
insrt = "insert into 591_xgb.auto_view values \
        ('A000','A000', 'code1', 1, 1, now()+3, 1, 1, 1, 3, 3, 3, 3),\
        ('A000','A000', 'code1', 1, 1, now()+3, 1, 1, 1, 3, 3, 3, 3), \
        ('A001','A001', 'code1', 1, 1, now()+3, 1, 1, 1, 3, 3, 3, 3);"
pd.read_sql(insrt,conn)

""


In [755]:
pd.read_sql('select * from 591_xgb.auto_view',conn)


,COOKIE,USER_ID,ITEM_ID,REGION_ID,SECTION_ID,TIMESTAMP,ACTION,CLICK,STAY,DIAL,COLLECT,FRIEND,SHARE
0,A000,A000,code1,1,1,1610075383,0,1,1,3,3,3,3
1,A000,A000,code1,1,1,1610075383,1,1,1,3,3,3,3
2,A001,A001,code1,1,1,1610075383,1,1,1,3,3,3,3


In [3]:
# pd.read_sql('optimize table 591_xgb.auto_view DEDUPLICATE;',conn)
pd.read_sql('OPTIMIZE TABLE 591_xgb.user_image FINAL DEDUPLICATE;',conn)

""


In [675]:
import time
time.time()

1610075177.7020912

# 增加欄位

In [4]:
pd.read_sql("ALTER TABLE 591_xgb.user_image ADD COLUMN COMMUNITY String AFTER ENCODE_parking_type",conn-)

""


# 修改欄位

In [13]:
# cols = pd.read_sql("SELECT * FROM 591_xgb.user_image limit 1",conn)
for col in cols.columns:
    if 'ohe' in col:
        sql = "ALTER TABLE 591_xgb.user_image MODIFY COLUMN %s Float32" % col
        print(sql)
#         pd.read_sql(sql,conn)
        sql = "ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN %s Float32" % col
        print(sql)
#         pd.read_sql(sql,conn)

ALTER TABLE 591_xgb.user_image MODIFY COLUMN PRICE_ohe1 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN PRICE_ohe1 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN PRICE_ohe2 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN PRICE_ohe2 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN PRICE_ohe3 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN PRICE_ohe3 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN PRICE_ohe4 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN PRICE_ohe4 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN PRICE_ohe5 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN PRICE_ohe5 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN PRICE_ohe6 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN PRICE_ohe6 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN PRICE_ohe7 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN PRICE_ohe7 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN PRICE_ohe8 Float3

ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN BALCONY_ohe3 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN BALCONY_ohe4 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN BALCONY_ohe4 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN AGE_ohe1 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN AGE_ohe1 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN AGE_ohe2 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN AGE_ohe2 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN AGE_ohe3 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN AGE_ohe3 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN AGE_ohe4 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN AGE_ohe4 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN AGE_ohe5 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN AGE_ohe5 Float32
ALTER TABLE 591_xgb.user_image MODIFY COLUMN AGE_ohe6 Float32
ALTER TABLE 591_xgb.user_image_train MODIFY COLUMN AGE_ohe6 Float32
ALTER TABL